In [55]:
import pulp
import pandas as pd
import json
import joblib
from rapidfuzz import fuzz, process
pd.set_option('display.max_columns', None)

In [56]:
df_ = pd.read_csv(f'../../data/fpl/gw-1/fpl_2425_updated_features.csv')
df_.head()

,name,position,team,opponent_team,total_points,was_home,team_tot_points_last_5_matches,team_tot_points_last_5_h2h,tot_assists_last_5_h2h,tot_bonus_last_5_h2h,tot_bps_last_5_h2h,tot_clean_sheets_last_5_h2h,tot_goals_conceded_last_5_h2h,tot_goals_scored_last_5_h2h,tot_own_goals_last_5_h2h,tot_penalties_missed_last_5_h2h,tot_penalties_saved_last_5_h2h,tot_red_cards_last_5_h2h,tot_yellow_cards_last_5_h2h,tot_saves_last_5_h2h,tot_total_points_last_5_h2h,avg_ict_index_last_5_h2h,avg_influence_last_5_h2h,avg_creativity_last_5_h2h,avg_threat_last_5_h2h,avg_selected_last_5_h2h,avg_minutes_last_5_h2h,tot_assists_last_5,tot_bonus_last_5,tot_bps_last_5,tot_clean_sheets_last_5,tot_goals_conceded_last_5,tot_goals_scored_last_5,tot_own_goals_last_5,tot_penalties_missed_last_5,tot_penalties_saved_last_5,tot_red_cards_last_5,tot_yellow_cards_last_5,tot_saves_last_5,tot_total_points_last_5,avg_ict_index_last_5,avg_influence_last_5,avg_creativity_last_5,avg_threat_last_5,avg_selected_last_5,avg_minutes_last_5
0,Aaron Cresswell,DEF,West Ham United,Aston Villa,0,True,0.0,4.0,0,0,3,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,107954.0,1.0,0,0,18,0,4,0,0,0,0,0,0,0,3,1.0,3.0,2.0,0.0,69082.0,22.0
1,Aaron Hickey,DEF,Brentford,Crystal Palace,0,True,3.0,3.0,0,0,37,0,4,0,0,0,0,0,2,0,3,2.0,9.0,3.0,4.0,22298.0,45.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,12001.0,0.0
2,Aaron Ramsdale,GK,Arsenal,Wolverhampton Wanderers,0,True,15.0,15.0,0,0,28,1,1,0,0,0,0,0,0,1,8,0.0,3.0,0.0,0.0,852736.0,36.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,485799.0,0.0
3,Aaron Wan-Bissaka,DEF,West Ham United,Aston Villa,0,True,0.0,3.0,0,0,17,0,2,0,0,0,0,0,0,0,1,0.0,3.0,0.0,0.0,153110.0,18.0,0,0,62,1,7,0,0,0,0,0,0,0,9,3.0,14.0,10.0,2.0,205004.0,71.0
4,Abdoulaye Doucouré,MID,Everton,Brighton & Hove Albion,2,True,0.0,4.0,0,2,56,1,5,2,0,0,0,0,2,0,21,5.0,19.0,15.0,20.0,41023.0,84.0,0,0,34,2,5,1,0,0,0,0,1,0,15,4.0,10.0,8.0,18.0,53500.0,73.0


In [57]:
df_['team'].sort_values().unique()

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford',
       'Brighton & Hove Albion', 'Chelsea', 'Crystal Palace', 'Everton',
       'Fulham', 'Ipswich Town', 'Leicester City', 'Liverpool',
       'Manchester City', 'Manchester United', 'Newcastle United',
       'Nottingham Forest', 'Southampton', 'Tottenham Hotspur',
       'West Ham United', 'Wolverhampton Wanderers'], dtype=object)

In [58]:
df_[df_['team'] == 'Arsenal']['name'].sort_values().unique()

array(['Aaron Ramsdale', 'Ben White', 'Bukayo Saka', 'David Raya',
       'Declan Rice', 'Eddie Nketiah', 'Ethan Nwaneri',
       'Fábio Ferreira Vieira', 'Gabriel Jesus', 'Gabriel Magalhães',
       'Gabriel Martinelli', 'Jakub Kiwior', 'Jorge Luiz Frello Filho',
       'Jurriën Timber', 'Kai Havertz', 'Karl Hein', 'Kieran Tierney',
       'Leandro Trossard', 'Martin Ødegaard', 'Myles Lewis-Skelly',
       'Oleksandr Zinchenko', 'Reiss Nelson', 'Riccardo Calafiori',
       'Takehiro Tomiyasu', 'Thomas Partey', 'William Saliba'],
      dtype=object)

In [59]:
def fpl_selection(gw):
    df = pd.read_csv(f'../../data/fpl/gw-{gw}/fpl_2425_updated_features.csv')

    def manual_min_max_scaling(value, min_val, max_val):
        return (value - min_val) / (max_val - min_val)

    # Load file JSON yang berisi nilai min dan max untuk setiap kolom
    with open('../../data/json/min_max_values.json', 'r') as json_file:
        min_max_dict = json.load(json_file)
        
    for column in min_max_dict.keys():
        min_val = min_max_dict[column]['min']
        max_val = min_max_dict[column]['max']
        df[column] = df[column].apply(
            lambda x: manual_min_max_scaling(x, min_val, max_val)
        )
        
    df.drop('total_points', axis=1, inplace=True)
    
    with open('../../data/json/encoded_player_names.json', 'r') as json_file:
        player_dict = json.load(json_file)
    with open('../../data/json/encoded_position.json', 'r') as json_file:
        position_dict = json.load(json_file)
    with open('../../data/json/encoded_team_name.json', 'r') as json_file:
        team_dict = json.load(json_file)
        
    df['name'] = df['name'].map(player_dict)
    df['position'] = df['position'].map(position_dict)
    df['team'] = df['team'].map(team_dict)
    df['opponent_team'] = df['opponent_team'].map(team_dict)
    
    model_lgbm = joblib.load('../../model/lgbm_model.joblib') 
    
    predicted_tot_points = model_lgbm.predict(df)
    df['predicted_total_points'] = predicted_tot_points
    
    def inverse_min_max_scaling(scaled_value, min_val, max_val):
        return (scaled_value * (max_val - min_val)) + min_val

    columns_to_inverse = [col for col in min_max_dict.keys() if col != 'total_points']

    for column in columns_to_inverse:
        min_val = min_max_dict[column]['min']
        max_val = min_max_dict[column]['max']
        df[column] = df[column].apply(
            lambda x: inverse_min_max_scaling(x, min_val, max_val)
        )
    
    min_val = min_max_dict['total_points']['min']
    max_val = min_max_dict['total_points']['max']
    df['predicted_total_points'] = df['predicted_total_points'].apply(lambda x: inverse_min_max_scaling(x, min_val, max_val))
    df['predicted_total_points'] = round(df['predicted_total_points'], 2)
    
        
    inverse_player_dict = {v: k for k, v in player_dict.items()}
    inverse_position_dict = {v: k for k, v in position_dict.items()}
    inverse_team_dict = {v: k for k, v in team_dict.items()}

    # Replace the columns with the original values
    df['name'] = df['name'].map(inverse_player_dict)
    df['position'] = df['position'].map(inverse_position_dict)
    df['team'] = df['team'].map(inverse_team_dict)
    df['opponent_team'] = df['opponent_team'].map(inverse_team_dict)
    
    
    fpl_price_df_gw = pd.read_csv(f'../../data/fpl/price/fpl-price-gw-{gw}.csv')

    team_map = {
        'Arsenal':'Arsenal',
        'Aston Villa':'Aston Villa',
        'Brentford':'Brentford',
        'Bournemouth':'Bournemouth',
        'Brighton':'Brighton & Hove Albion',
        'Burnley':'Burnley',
        'Chelsea':'Chelsea',
        'Cardiff':'Cardiff City',
        'Crystal Palace':'Crystal Palace',
        'Everton':'Everton',
        'Hull':'Hull City',
        'Huddersfield':'Huddersfield Town',
        'Leeds':'Leeds United',
        'Leicester':'Leicester City',
        'Ipswich': 'Ipswich Town',
        'Liverpool':'Liverpool',
        'Luton':'Luton Town',
        'Man City':'Manchester City',
        'Man Utd':'Manchester United',
        'Middlesbrough':'Middlesbrough',
        'Newcastle':'Newcastle United',
        'Norwich':'Norwich City',
        "Nott'm Forest":'Nottingham Forest',
        'Southampton':'Southampton',
        'Swansea':'Swansea City',
        'Stoke':'Stoke City',
        'Sunderland':'Sunderland',
        'Sheffield Utd':'Sheffield United',
        'Spurs':'Tottenham Hotspur',
        'Watford':'Watford',
        'West Ham':'West Ham United',
        'West Brom':'West Bromwich United',
        'Wolves':'Wolverhampton Wanderers'
    }

    fpl_price_df_gw['team'] = fpl_price_df_gw['team'].replace(team_map)

    pulp_df = df[['name', 'position', 'team', 'opponent_team', 'predicted_total_points']]
    
    def find_best_match(row, players_df):
        # Filter players_df berdasarkan 'team' dan 'position'
        subset = players_df[
            (players_df['team'] == row['team']) &
            (players_df['position'] == row['position'])
        ]
        # Jika ada lebih dari satu kandidat, cari yang paling cocok
        if not subset.empty:
            best_match = process.extractOne(row['name'], subset['name'], scorer=fuzz.token_sort_ratio)
            if best_match and best_match[1] > 30:  # Threshold untuk kecocokan
                return subset.loc[subset['name'] == best_match[0], 'price'].values[0]
        return None  # Kembalikan None jika tidak ada kecocokan

    # Terapkan fungsi untuk menambahkan kolom 'price' ke pulp_df
    pulp_df['price'] = pulp_df.apply(find_best_match, axis=1, players_df=fpl_price_df_gw)
    
    return pulp_df

In [60]:
pulp_df = fpl_selection(gw=1)

[LightGBM] [Warning] min_data_in_leaf is set=79, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=79
[LightGBM] [Warning] feature_fraction is set=0.9635863767391594, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9635863767391594
[LightGBM] [Warning] bagging_fraction is set=0.9708055923470893, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9708055923470893
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


C:\Users\marse\AppData\Local\Temp\ipykernel_45060\1293585817.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pulp_df['price'] = pulp_df.apply(find_best_match, axis=1, players_df=fpl_price_df_gw)


In [61]:
def find_condition(row, gw=1):
    fpl_price_info = pd.read_csv(f'../../data/fpl/combined-fpl-price.csv')
    fpl_price_info = fpl_price_info[fpl_price_info['GW'] == gw]
    
    # Mencari kesamaan nama dengan threshold diatas 60
    match = process.extractOne(row['name'], fpl_price_info['name'], score_cutoff=60)
    
    if match:
        matched_name = match[0]
        # Ambil baris yang sesuai dengan nama yang cocok
        matched_row = fpl_price_info[(fpl_price_info['name'] == matched_name) & 
                                #    (fpl_price_info['team'] == row['team']) & 
                                   (fpl_price_info['position'] == row['position'])]
        
        if not matched_row.empty:
            # Kembalikan nilai kondisi jika ditemukan
            return matched_row['condition'].values[0]
    return 1

# Menambahkan kolom condition berdasarkan kecocokan
pulp_df['condition'] = pulp_df.apply(find_condition, axis=1)

In [62]:
pulp_df

,name,position,team,opponent_team,predicted_total_points,price,condition
0,Aaron Cresswell,DEF,West Ham United,Aston Villa,0.61,4.0,0.25
1,Aaron Hickey,DEF,Brentford,Crystal Palace,-0.07,4.5,0.00
2,Aaron Ramsdale,GK,Arsenal,Wolverhampton Wanderers,-0.03,4.4,1.00
3,Aaron Wan-Bissaka,DEF,West Ham United,Aston Villa,0.63,4.5,1.00
4,Abdoulaye Doucouré,MID,Everton,Brighton & Hove Albion,3.06,5.4,1.00
...,...,...,...,...,...,...,...
611,Yunus Konak,MID,Brentford,Crystal Palace,0.01,4.5,1.00
612,Yves Bissouma,MID,Tottenham Hotspur,Leicester City,0.59,5.0,1.00
613,Álex Moreno Lopera,DEF,Aston Villa,West Ham United,0.01,4.5,1.00
614,Ângelo Gabriel Borges Damaceno,MID,Chelsea,Manchester City,0.01,4.5,1.00


In [63]:
import pulp

# Define the problem
problem = pulp.LpProblem("Best_Starting_Eleven", pulp.LpMaximize)

# Define decision variables
player_vars = pulp.LpVariable.dicts("Players", pulp_df.index, cat='Binary')

# Objective function: Maximize predicted total points
problem += pulp.lpSum([player_vars[i] * pulp_df.loc[i, 'predicted_total_points'] for i in pulp_df.index]), "Total_Points"

# Constraint 1: Total 15 players
problem += pulp.lpSum([player_vars[i] for i in pulp_df.index]) == 30, "Total_15_Players"

# Constraint 2: Max total price is 100 pounds
# problem += pulp.lpSum([player_vars[i] * pulp_df.loc[i, 'price'] for i in pulp_df.index]) <= 100, "Max_Total_Price"

# Constraint 3: Max 3 players from the same team
for team in pulp_df['team'].unique():
    problem += pulp.lpSum([player_vars[i] for i in pulp_df.index if pulp_df.loc[i, 'team'] == team]) <= 3, f"Max_3_Players_from_{team}"

# Constraint 4: Min 3 DEF, MID, and FWD
for position in ['DEF', 'MID', 'FWD']:
    problem += pulp.lpSum([player_vars[i] for i in pulp_df.index if pulp_df.loc[i, 'position'] == position]) >= 3, f"Min_3_{position}"

# Constraint 5: Exactly 2 GK (Goalkeepers)
problem += pulp.lpSum([player_vars[i] for i in pulp_df.index if pulp_df.loc[i, 'position'] == 'GK']) == 4, "Exactly_3_GK"

# Constraint 6: Min 4 players from each position group: DEF, MID, and FWD
for position in ['DEF', 'MID', 'FWD']:
    problem += pulp.lpSum([player_vars[i] for i in pulp_df.index if pulp_df.loc[i, 'position'] == position]) >= 4, f"Min_4_{position}_Total"

# Solve the problem
problem.solve()

1

In [65]:
#  Extract the selected players
selected_players = pulp_df.loc[[i for i in pulp_df.index if player_vars[i].varValue == 1]]

# Check for duplicates
if selected_players.duplicated(subset=['name']).any():
    print("Warning: Duplicate players detected.")
else:
    print("No duplicate players detected.")

# Define the desired order of positions
position_order = {'GK': 1, 'DEF': 2, 'MID': 3, 'FWD': 4}

# Add an auxiliary column to the DataFrame to represent the order
selected_players['position_order'] = selected_players['position'].map(position_order)

# Sort the DataFrame by the auxiliary column and drop it afterward
sorted_players = selected_players.sort_values(by='position_order').drop(columns='position_order')

print("Prediction Selected Starting Eleven - Gameweek 2")
sorted_players[['name', 'team', 'opponent_team', 'predicted_total_points', 'price', 'position']]

No duplicate players detected.
Prediction Selected Starting Eleven - Gameweek 2


,name,team,opponent_team,predicted_total_points,price,position
27,Alisson Becker,Liverpool,Ipswich Town,5.65,5.5,GK
37,André Onana,Manchester United,Fulham,6.18,5.0,GK
454,Nick Pope,Newcastle United,Southampton,6.31,5.0,GK
139,David Raya,Arsenal,Wolverhampton Wanderers,6.31,5.5,GK
484,Pedro Porro,Tottenham Hotspur,Leicester City,7.03,5.5,DEF
463,Noussair Mazraoui,Manchester United,Fulham,8.36,4.5,DEF
445,Nathan Collins,Brentford,Crystal Palace,5.09,4.5,DEF
313,Joël Veltman,Brighton & Hove Albion,Everton,7.05,4.5,DEF
5,Abdul Fatawu,Leicester City,Tottenham Hotspur,7.39,5.5,MID
523,Ryan Yates,Nottingham Forest,Bournemouth,7.62,5.0,MID
